In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Load dataset
digits = load_digits()
X = digits.images / 16.0
y = digits.target.reshape(-1, 1)

# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y)

# Flatten (8x8 → 64 features)
X = X.reshape(len(X), -1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

In [3]:
# Bigger model (~19k params)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation="relu", input_shape=(64,)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [4]:
history = model.fit(X_train, y_train, epochs=30, batch_size=32,
                    validation_split=0.1, verbose=1)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}")

model.summary()

Epoch 1/30
41/41 [==============================] - 2s 8ms/step - loss: 1.8171 - accuracy: 0.4671 - val_loss: 1.3512 - val_accuracy: 0.6667
Epoch 2/30
41/41 [==============================] - 0s 5ms/step - loss: 0.8945 - accuracy: 0.7927 - val_loss: 0.5654 - val_accuracy: 0.8611
Epoch 3/30
41/41 [==============================] - 0s 4ms/step - loss: 0.4350 - accuracy: 0.9010 - val_loss: 0.3261 - val_accuracy: 0.9097
Epoch 4/30
41/41 [==============================] - 0s 4ms/step - loss: 0.2820 - accuracy: 0.9335 - val_loss: 0.2646 - val_accuracy: 0.9097
Epoch 5/30
41/41 [==============================] - 0s 4ms/step - loss: 0.2081 - accuracy: 0.9513 - val_loss: 0.2237 - val_accuracy: 0.9167
Epoch 6/30
41/41 [==============================] - 0s 4ms/step - loss: 0.1640 - accuracy: 0.9652 - val_loss: 0.1806 - val_accuracy: 0.9444
Epoch 7/30
41/41 [==============================] - 0s 4ms/step - loss: 0.1575 - accuracy: 0.9575 - val_loss: 0.1693 - val_accuracy: 0.9306
Epoch 8/30
41/41 [==

In [5]:
# Representative dataset for int8 quantization
def representative_dataset():
    for i in range(len(X_train)):
        yield [X_train[i:i+1].astype(np.float32)]

# Convert to fully int8
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model = converter.convert()

# Save fully int8 TFLite model
with open("digits_model_19k.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\kukil\AppData\Local\Temp\tmp_bgttq60\assets


C:\Users\kukil\.conda\envs\tinyML\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [6]:
!xxd -i digits_model_quant_19k.tflite > digits_model_quant_19k.h

In [7]:
interpreter = tf.lite.Interpreter(model_path="digits_model_19k.tflite")
interpreter.allocate_tensors()

for detail in interpreter.get_tensor_details():
    print(detail['name'], detail['dtype'])


serving_default_dense_input:0 <class 'numpy.int8'>
sequential/dense_3/BiasAdd/ReadVariableOp <class 'numpy.int32'>
sequential/dense_3/MatMul <class 'numpy.int8'>
sequential/dense_2/BiasAdd/ReadVariableOp <class 'numpy.int32'>
sequential/dense_2/MatMul <class 'numpy.int8'>
sequential/dense_1/BiasAdd/ReadVariableOp <class 'numpy.int32'>
sequential/dense_1/MatMul <class 'numpy.int8'>
sequential/dense/BiasAdd/ReadVariableOp <class 'numpy.int32'>
sequential/dense/MatMul <class 'numpy.int8'>
sequential/dense/MatMul;sequential/dense/Relu;sequential/dense/BiasAdd <class 'numpy.int8'>
sequential/dense_1/MatMul;sequential/dense_1/Relu;sequential/dense_1/BiasAdd <class 'numpy.int8'>
sequential/dense_2/MatMul;sequential/dense_2/Relu;sequential/dense_2/BiasAdd <class 'numpy.int8'>
sequential/dense_3/MatMul;sequential/dense_3/BiasAdd <class 'numpy.int8'>
StatefulPartitionedCall:0 <class 'numpy.int8'>
